In [1]:
from pipeline import Pipeline, Process, Postprocess, Preprocess
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
class MyPreProcess(Preprocess):
    process_desc = "Exemple de pre-process -> ne fait rien"
    def run(self, images):
        pass


class MyProcess(Process):
    process_desc = "Exemple de process -> ne fait rien"
    def run(self, images):
        pass
    
    
class MyPostProcess(Postprocess):
    process_desc = "Exemple de post-process -> ne fait rien"
    def run(self, images):
        pass

In [3]:
pipeline = Pipeline()
process_array = np.array([MyPreProcess()
                          ,MyProcess()
                          ,MyPostProcess()
                          ,MyPreProcess(verbose=0)])
pipeline.add_processes(process_array)

Preprocess :  Exemple de pre-process -> ne fait rien
Process :  Exemple de process -> ne fait rien
Postprocess :  Exemple de post-process -> ne fait rien
Exemple de pre-process -> ne fait rien a été ajouté.
Exemple de process -> ne fait rien a été ajouté.
Exemple de post-process -> ne fait rien a été ajouté.
Exemple de pre-process -> ne fait rien a été ajouté.


In [4]:
pipeline.print_process()

Exemple de pre-process -> ne fait rien
Exemple de pre-process -> ne fait rien
Exemple de process -> ne fait rien
Exemple de post-process -> ne fait rien


In [5]:
data = pd.read_csv('./../data/SEATGURU_INFO_AIRCRAFT.csv', sep = ';')
data.index = range(2, len(data) + 2)

In [6]:
data.head()

,Data_Source,Airline_name,Aircraft_Type,Category,Flight_Type,Count,Total_seat,Seat_Class,Seat_Type,Seat_Position,...,Width,Bedlength_maximum,Bedlength_minimum,Music,VideoType,Power_available,Power_kind,Usb_available,Usb_kind,Rating
2,SeatGuru,aegean-airlines,Airbus A319-100,ECONOMY,short haul,138,138,Economy,STANDARD,BULKHEAD,...,18.0,NaN,NaN,NONE,NONE,NONE,NaN,NONE,NaN,2
3,SeatGuru,aegean-airlines,Airbus A319-100,ECONOMY,short haul,138,138,Economy,STANDARD,NORMAL,...,18.0,NaN,NaN,NONE,NONE,NONE,NaN,NONE,NaN,2
4,SeatGuru,aegean-airlines,Airbus A319-100,ECONOMY,short haul,138,138,Economy,STANDARD,EXIT_ROW,...,18.0,NaN,NaN,NONE,NONE,NONE,NaN,NONE,NaN,5
5,SeatGuru,aegean-airlines,Airbus A319-100,ECONOMY,short haul,138,138,Economy,STANDARD,NORMAL,...,18.0,NaN,NaN,NONE,NONE,NONE,NaN,NONE,NaN,3
6,SeatGuru,aegean-airlines,Airbus A319-100,ECONOMY,short haul,138,138,Economy,STANDARD,NORMAL,...,18.0,NaN,NaN,NONE,NONE,NONE,NaN,NONE,NaN,1


In [7]:
data.columns

Index(['Data_Source', 'Airline_name', 'Aircraft_Type', 'Category',
       'Flight_Type', 'Count', 'Total_seat', 'Seat_Class', 'Seat_Type',
       'Seat_Position', 'Recline', 'Premium', 'Pitch_minimum', 'Pitch_maximum',
       'Width', 'Bedlength_maximum', 'Bedlength_minimum', 'Music', 'VideoType',
       'Power_available', 'Power_kind', 'Usb_available', 'Usb_kind', 'Rating'],
      dtype='object')

### Ajouter image au DF

In [108]:
data['airline_aircraft'] = data.apply(lambda x: x['Airline_name'].replace('-', '_') + '_' + x['Aircraft_Type'].replace(' ', '_'), axis = 'columns')
liste_image = os.listdir('./../data/LAYOUT SEATGURU/')
l_result = []
for i in liste_image:
    for j in data['airline_aircraft'].unique():
        if j.lower() in i.lower():
            l_result.append((i, j))

new_df = pd.DataFrame(list(zip([i[0] for i in l_result], [i[1] for i in l_result])), columns =['image', 'avion'])

In [110]:
df_temp = new_df[['avion', 'image']]

In [111]:
df_temp = pd.DataFrame([list(data.groupby('airline_aircraft').groups.keys()), [list(i) for i in list(data.groupby('airline_aircraft').groups.values())]], index = ['avion', 'lignes']).T

In [112]:
df_temp = df_temp.sort_values('lignes').reset_index(drop = True)

In [114]:
new_df = new_df.merge(df_temp, how='outer', on='avion').dropna()

In [118]:
new_df.drop('avion', axis = 'columns', inplace = True)

In [119]:
new_df.head()

,image,lignes
0,Aegean_Airlines_Airbus_A319-100_plane1.svg.png,"[2, 3, 4, 5, 6]"
1,Aegean_Airlines_Airbus_A320-200_plane2.svg.png,"[7, 8, 9, 10, 11, 12]"
2,Aegean_Airlines_Airbus_A321-200_plane1.svg.png,"[13, 14, 15, 16, 17, 18, 19]"
3,Aegean_Airlines_Airbus_A321-200_plane3.svg.png,"[13, 14, 15, 16, 17, 18, 19]"
4,Aegean_Airlines_ATR_42-600_plane2.svg.png,"[20, 21, 22, 23, 24]"


In [120]:
# new_df.to_csv('./../data/Image_lignes.csv', sep = ';')

### Autre

In [10]:
path = './../data/LAYOUT SEATGURU/'
df = pd.DataFrame({'img': os.listdir('./../data/LAYOUT SEATGURU/'), 'chemin': [path + img_name for img_name in os.listdir('./../data/LAYOUT SEATGURU/')]})

In [11]:
df.head()

,img,chemin
0,Aegean_Airlines_Airbus_A319-100_plane1.svg.png,./../data/LAYOUT SEATGURU/Aegean_Airlines_Airb...
1,Aegean_Airlines_Airbus_A320-200_plane2.svg.png,./../data/LAYOUT SEATGURU/Aegean_Airlines_Airb...
2,Aegean_Airlines_Airbus_A321-200_plane1.svg.png,./../data/LAYOUT SEATGURU/Aegean_Airlines_Airb...
3,Aegean_Airlines_Airbus_A321-200_plane3.svg.png,./../data/LAYOUT SEATGURU/Aegean_Airlines_Airb...
4,Aegean_Airlines_ATR_42-600_plane2.svg.png,./../data/LAYOUT SEATGURU/Aegean_Airlines_ATR_...


In [1]:
img = plt.imread(df['chemin'][0])
figure = plt.figure(figsize = (5, 20))
plt.imshow(img)

NameError: name 'plt' is not defined

In [ ]:
cv2.destroyAllWindows()

In [22]:
d = pd.read_csv('./Image_lignes.csv', sep = ';')

In [23]:
d.head()

,Unnamed: 0,image,lignes
0,0,Aegean_Airlines_Airbus_A319-100_plane1.svg.png,"[2, 3, 4, 5, 6]"
1,1,Aegean_Airlines_Airbus_A320-200_plane2.svg.png,"[7, 8, 9, 10, 11, 12]"
2,2,Aegean_Airlines_Airbus_A321-200_plane1.svg.png,"[13, 14, 15, 16, 17, 18, 19]"
3,3,Aegean_Airlines_ATR_42-600_plane2.svg.png,"[20, 21, 22, 23, 24]"
4,4,Aegean_Airlines_Bombardier_Dash_8_Q400_plane3....,"[25, 26, 27]"


In [33]:
d

,Unnamed: 0,image,lignes
0,0,Aegean_Airlines_Airbus_A319-100_plane1.svg.png,"[2, 3, 4, 5, 6]"
1,1,Aegean_Airlines_Airbus_A320-200_plane2.svg.png,"[7, 8, 9, 10, 11, 12]"
2,2,Aegean_Airlines_Airbus_A321-200_plane1.svg.png,"[13, 14, 15, 16, 17, 18, 19]"
3,3,Aegean_Airlines_ATR_42-600_plane2.svg.png,"[20, 21, 22, 23, 24]"
4,4,Aegean_Airlines_Bombardier_Dash_8_Q400_plane3....,"[25, 26, 27]"
5,5,Aeroflot_Russian_Airlines_Airbus_330_200_plane...,"[28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]"
6,6,Aeroflot_Russian_Airlines_Airbus_A320_plane7.jpg,"[39, 40, 41, 42, 43, 44, 45, 46]"
7,7,Aeroflot_Russian_Airlines_Airbus_A320_V2_plane...,"[47, 48, 49, 50, 51, 52, 53]"
8,8,Aeroflot_Russian_Airlines_Airbus_A321_plane9.jpg,"[54, 55, 56, 57, 58, 59, 60, 61, 62]"
9,9,Aeroflot_Russian_Airlines_Airbus_A321_V2_plane...,"[63, 64, 65, 66, 67, 68, 69, 70, 71, 72]"
